In [1]:
import os
import sys
from pathlib import Path

import bpy
import mathutils
import numpy as np
import pickle

# Imports from this repository
sys.path.append("../../")
sys.path.append("../")
from simulation.cloth_3d_util.accessor import Cloth3DCanonicalAccessor
from simulation.cloth_3d_util.util import loadInfo
from simulation.pipeline.simulate_garment_hanging_rest_state import \
    simulate_garment_hanging_rest_state
from simulation.blender_util_dylan.physics import set_sim_output_as_default_mesh_shape
from simulation.blender_util_dylan.checkpointer import BlendFileCheckpointer
from simulation.blender_util_dylan.gripper import GripperAnimation
from simulation.blender_util_dylan.debug import print_obj_keyframe_coordinates

%load_ext autoreload
%autoreload 2

WARN (bgl): source/blender/python/generic/bgl.c:2654 BPyInit_bgl: 'bgl' imported without an OpenGL backend. Please update your add-ons to use the 'gpu' module. In Blender 4.0 'bgl' will be removed.


/home/dcolli23/.local/share/virtualenvs/simulation-7NaVsHdv/lib/python3.10/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/dcolli23/.local/share/virtualenvs/simulation-7NaVsHdv/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/dcolli23/.local/share/virtualenvs/simulation-7NaVsHdv/lib/python3.10/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/dcolli23/.local/share/virtualenvs/simulation-7NaVsHdv/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  retur

## Dynamics Checkpointing: Simulate Or Load Hanging Resting State

Slightly modified the checkpointing routine to also save/load the resting state simulation (SMPL simulation) results dictionary to/from a pickle file.

In [2]:
FILE_ROOT = Path(os.getcwd())
CLOTH3D_PATH = Path(os.path.expanduser("~/DataLocker/datasets/CLOTH3D/training/"))
OUTPUT_ROOT = FILE_ROOT / ".." / "script_output" / "sim_pipeline_driver_test"

# Make the output directory if it doesn't exist.
OUTPUT_ROOT.mkdir(exist_ok=True)


PLANE_OFFSET = 0.025  # [m]

In [3]:
sample_configuration = {
    "sample_id": "00380",
    "garment_name": "Tshirt",
    "grip_vertex_idx": 0
}

smpl_simulation_duration_pair = (0, 120)

sample_configs = [sample_configuration]

In [4]:
## Setup the simulation.
accessor = Cloth3DCanonicalAccessor(CLOTH3D_PATH)
config = sample_configs[0]
sample_key = f"{config['sample_id']}_{config['garment_name']}_{config['grip_vertex_idx']}_rendering_prototyping"
sample_dir = OUTPUT_ROOT / sample_key
sample_dir.mkdir(exist_ok=True)

result_file = "simulation_result.pk"
result_path = sample_dir / result_file

# Get a dictionary containing the data for this sample garment.
sample_data = accessor.get_sample_data(**config)

garment_info_mat_filename = CLOTH3D_PATH / config["sample_id"] / "info.mat"
garment_info = loadInfo(garment_info_mat_filename)

checkpointer = BlendFileCheckpointer(sample_dir, save_new_checkpoints=True)

In [5]:
if checkpointer.does_rest_state_checkpoint_exist():
    print("Hanging state checkpoint exists. Loading checkpoint from file.")
    # Load the checkpoint instead of simulating the resting state again.
    result_data_smpl = checkpointer.load_hanging_rest_state()
else:
    print("Hanging state checkpoint does not exist. Simulating and saving checkpoint.")
    # Run the SMPL simulation to get the garment in a stationary hanging configuration
    frames_to_resting_state = 120
    result_data_smpl = simulate_garment_hanging_rest_state(config, sample_data, frames_to_resting_state)

    # Then set the default mesh shape to this shape
    cloth_obj = bpy.data.objects["cloth"]
    set_sim_output_as_default_mesh_shape(cloth_obj, frames_to_resting_state)

    # Now create a checkpoint for this so we don't have to resimulate every time we run this
    # notebook.
    checkpointer.save_hanging_rest_state(result_data_smpl=result_data_smpl)

Hanging state checkpoint does not exist. Simulating and saving checkpoint.
Running SMPL Simulation Pipeline
--------------------------------------------------------------------------------
bake: frame 0 :: 120
bake: frame 1 :: 120
bake: frame 2 :: 120
bake: frame 3 :: 120
bake: frame 4 :: 120
bake: frame 5 :: 120
bake: frame 6 :: 120
bake: frame 7 :: 120
bake: frame 8 :: 120
bake: frame 9 :: 120
bake: frame 10 :: 120
bake: frame 11 :: 120
bake: frame 12 :: 120
bake: frame 13 :: 120
bake: frame 14 :: 120
bake: frame 15 :: 120
bake: frame 16 :: 120
bake: frame 17 :: 120
bake: frame 18 :: 120
bake: frame 19 :: 120
bake: frame 20 :: 120
bake: frame 21 :: 120
bake: frame 22 :: 120
bake: frame 23 :: 120
bake: frame 24 :: 120
bake: frame 25 :: 120
bake: frame 26 :: 120
bake: frame 27 :: 120
bake: frame 28 :: 120
bake: frame 29 :: 120
bake: frame 30 :: 120
bake: frame 31 :: 120
bake: frame 32 :: 120
bake: frame 33 :: 120
bake: frame 34 :: 120
bake: frame 35 :: 120
bake: frame 36 :: 120
bake: f

In [6]:
from simulation.blender_util_dylan.debug import print_nested_dict_types

print_nested_dict_types(result_data_smpl)


cloth_state
	uv_faces <class 'list'>
	uv_verts <class 'numpy.ndarray'>
	faces <class 'list'>
	edges <class 'numpy.ndarray'>
	verts <class 'numpy.ndarray'>
grip_vertex_idx <class 'int'>


In [7]:
# Just looking at the data in the results dictionary
# result_data_smpl['cloth_state']['uv_faces']
# result_data_smpl['cloth_state']['uv_verts']
# result_data_smpl['cloth_state']['faces']

## Try Rendering The SMPL Simulation Output

The saved results dictionary is simply the `cloth_state` (output of `mesh_to_numpy()`) and the `grip_vertex_idx` stored in a dictionary.

Cheng's code for the rendering pipeline appears to be in `blender_experiment/pipeline/smpl_rendering_pipeline.py`. I should (hopefully) be able to yank a lot of this code without modfification and use it.

In [8]:
from simulation.blender_util.camera import (
    require_camera, 
    get_camera_extrinsic,
    set_camera_extrinsic,
    get_camera_intrinsic, 
    set_camera_intrinsic,
    set_camera_focus_point,
    generate_intrinsic
)
from simulation.blender_util.physics import require_subdiv_modifier
from simulation.blender_util.material import (
    require_material,
    clear_materials,
    get_world_material,
    setup_black_world_material,
    setup_hdri_world_material,
    setup_metal_materail, 
    setup_plastic_material, 
    setup_uv_material, 
    setup_white_material,
    setup_textured_bsdf_material,
    require_image
)
from simulation.blender_util.render import (
    setup_color_management_raw,
    setup_color_management_srgb,
    setup_cycles,
    setup_eevee,
    get_cycles_uviz_config,
    get_eevee_rgb_config,
    get_cycles_rgb_config,
    setup_png_output,
    setup_exr_output,
    render_animation
)
from simulation.cloth_3d_util.accessors.access_functions import (
    get_info, get_garment_texture, get_garment_metadata
)
from simulation.blender_util.mesh import (NumpyBMeshManager, require_mesh, set_material)
from simulation.blender_util.compositor import (
    setup_trivial_compositor, 
    setup_uviz_compositor
)
from simulation.blender_util.collection import remove_all_collections

### Setup Debug Variables

A lot of these are ultimately going to be arguments passed to the rendering pipeline function

In [9]:
# import os
# os.chdir(os.path.expanduser("~/dev/blender_experiment"))

dataset_path = CLOTH3D_PATH
simulation_dir = sample_dir
simulation_result_path = simulation_dir.joinpath('simulation_result.pk')
simulation_result = result_data_smpl

# TODO: Seems like this should be read from the SMPL garment info we read at the top of the notebook
sample_id, garment_name, grip_vertex_idx = simulation_dir.stem.split('_')[:-2]
grip_vertex_idx = int(grip_vertex_idx)

cloth_state = simulation_result['cloth_state']

garment_verts = cloth_state['verts']
garment_faces = cloth_state['faces']
garment_uv_verts = cloth_state['uv_verts']
garment_uv_faces = cloth_state['uv_faces']

output_path = str(simulation_dir.absolute())
print("output path:", output_path)

info = get_info(dataset_path, sample_id)
garment_texture = get_garment_texture(dataset_path, sample_id, garment_name, info=info)
garment_meta = get_garment_metadata(dataset_path, sample_id, garment_name, info=info)
gender = garment_meta['gender']
fabric = garment_meta['garment_fabric']

num_camera_angles = 4
camera_intrinsic = generate_intrinsic(1024, 1024, 2048)

# %%
# enable GPU
cprefs = bpy.context.preferences.addons['cycles'].preferences
cprefs.compute_device_type = 'CUDA'
cprefs.get_devices()
for device in cprefs.devices:
    if device.type == 'CUDA':
        device.use = True
    else:
        device.use = False
    print(device.id, device.use, device.type)

# %%
# test
# import time
# s = time.perf_counter()
# smpl_cloth_pipeline(
# # io
# output_path,
# # metadata
# sample_id, garment_name, gender, fabric,
# # garment data
# garment_verts, garment_faces, 
# garment_uv_verts, garment_uv_faces, 
# garment_texture,
# # camera setting
# num_camera_angles,
# camera_intrinsic
# )
# print("Total time:", time.perf_counter() - s)

output path: /home/dcolli23/code/school/rob599_deeprob/projects/final/garmentnets_tracking/simulation/runners/../script_output/sim_pipeline_driver_test/00380_Tshirt_0_rendering_prototyping
CUDA_NVIDIA GeForce RTX 2080 Ti_0000:01:00 True CUDA
CPU False CPU
CUDA_NVIDIA GeForce RTX 2080 Ti_0000:01:00_OptiX False OPTIX


In [10]:
# I might eventually want to just load checkpoints and render from checkpoints since that will 
# probably be easier than using Cheng's interface but for just making sure everything works I'll use
# his methods.

using_chengs_checkpoint_method = True

if using_chengs_checkpoint_method:
    # clear existing state
    remove_all_collections()

    # %%
    # load mesh
    cloth_obj = require_mesh('cloth')
    with NumpyBMeshManager(
        verts=garment_verts, faces=garment_faces, 
        uv=garment_uv_verts, uv_faces=garment_uv_faces) as bm:
        bm.to_mesh(cloth_obj.data)
else:
    # NOTE: Assuming we're starting from a saved checkpoint instead of using Cheng's BMesh 
    # checkpointing.

    # Grab the cloth object
    cloth_obj = bpy.data.objects['cloth']    

    raise NotImplementedError()

In [11]:


# Setup the camera.
camera_obj = require_camera()
set_camera_intrinsic(camera_obj, camera_intrinsic)

z_offset = -0.8
radius = 4.0
focus_point = np.array([0,0,z_offset])
camera_rads = np.linspace(0, 2*np.pi, num=num_camera_angles+1)[:-1]
# TODO: I thought Cheng said he used 4 angles, not 3?
camera_locations = np.zeros((num_camera_angles, 3))
camera_locations[:, 0] = np.cos(camera_rads)
camera_locations[:, 1] = np.sin(camera_rads)
camera_locations *= radius
camera_locations[:, 2] = z_offset

camera_extrinsic_list = list()
for i in range(len(camera_locations)):
    camera_obj.location = camera_locations[i]
    set_camera_focus_point(camera_obj, focus_point)
    extrinsic = get_camera_extrinsic(camera_obj)
    camera_extrinsic_list.append(extrinsic)

In [12]:
# generate output filenames
output_dir = Path(output_path)
pickle_path = str(output_dir.joinpath('meta.pk').absolute())
uviz_paths = list()
rgb_paths = list()
for i in range(num_camera_angles):
    uviz_paths.append(str(output_dir.joinpath(
        'uviz_{}.exr'.format(i)).absolute()))
    rgb_paths.append(str(output_dir.joinpath(
        'rgb_{}.png'.format(i)).absolute()))
# curr_s = time.perf_counter()
# print("Setup: {}".format(curr_s - s))
# s = curr_s

In [13]:
# setup modifiers for rendering
require_subdiv_modifier(cloth_obj)

#### Setup Materials

These materials seem to be okay?

In [14]:
clear_materials()

In [15]:
uv_material = require_material('uv')
setup_uv_material(uv_material)

In [16]:
image = require_image('texture.png', garment_texture)
cloth_material = require_material('rgb')
setup_textured_bsdf_material(cloth_material, image)

In [17]:
checkpointer.save_checkpoint_if_desired()

Info: Saved "checkpoint_1.blend"
Saved blend file checkpoint to '/home/dcolli23/code/school/rob599_deeprob/projects/final/garmentnets_tracking/simulation/runners/../script_output/sim_pipeline_driver_test/00380_Tshirt_0_rendering_prototyping/checkpoint_1.blend'


1

#### Render Images

I'm having trouble with the setup from Cheng's code since I believe it was Blender version 2.X and I'm using version 3.5. I'm going to do some debugging below to figure this out.

In [18]:
from simulation.blender_util.render import get_renderer_config, get_default_cycles_config

print(bpy.context.scene.view_layers.keys())

# The active scene is the only scene available so this shouldn't be the problem.
print(bpy.context.scene)
print(bpy.data.scenes.keys())

print("Render FPS:", bpy.context.scene.render.fps)
print("Render FPS base:", bpy.context.scene.render.fps_base)

print("Resolution x:,", bpy.context.scene.render.resolution_x)
print("Resolution y:,", bpy.context.scene.render.resolution_y)

default_render_config_blender = get_renderer_config(bpy.context.scene.render)
# print("Default Blender Render Config:")
# for k, v in default_render_config_blender.items():
#     print(f"\t{k} = {v}")



default_render_config_cheng = get_default_cycles_config()
# print("Default render config:")
# for k, v in default_render_config_blender.items():
#     print(f"\t{k} = {v}")

# settings_missing_from_cheng_config = set(default_render_config_blender) - set(default_render_config_cheng)
# settings_in_cheng_config_not_in_blender = set(default_render_config_cheng) - set(default_render_config_blender)

# print(settings_missing_from_cheng_config)

print("Settings missing from Cheng's config:")
for k, v in default_render_config_blender.items():
    if k not in default_render_config_cheng:
        print(f"\t{k} = {v}")
print()

# print("Settings in Cheng's config but NOT Blender:")
# for k, v in settings_in_cheng_config_not_in_blender:
#     print(f"\t{k} = {v}")

['ViewLayer']
<bpy_struct, Scene("Scene") at 0x41f6818>
['Scene']
Render FPS: 24
Render FPS base: 1.0
Resolution x:, 1024
Resolution y:, 1024
Settings missing from Cheng's config:
	resolution_x = 1024
	resolution_y = 1024
	resolution_percentage = 100
	preview_pixel_size = AUTO
	pixel_aspect_x = 1.0
	pixel_aspect_y = 1.0
	fps = 24
	fps_base = 1.0
	frame_map_old = 100
	frame_map_new = 100
	dither_intensity = 1.0
	filter_size = 1.5
	film_transparent = False
	use_freestyle = False
	threads = 12
	threads_mode = AUTO
	use_motion_blur = False
	motion_blur_shutter = 0.5
	hair_type = STRAND
	hair_subdiv = 0
	use_high_quality_normals = False
	use_border = False
	border_min_x = 0.0
	border_min_y = 0.0
	border_max_x = 1.0
	border_max_y = 1.0
	use_crop_to_border = False
	use_placeholder = False
	use_overwrite = True
	use_compositing = True
	use_sequencer = True
	use_file_extension = True
	use_lock_interface = False
	filepath = /tmp/
	use_render_cache = False
	use_bake_selected_to_active = False
	us

In [19]:
# uviz
world_material = get_world_material()
setup_black_world_material(world_material)
set_material(cloth_obj, uv_material)
# setup compositor
setup_uviz_compositor()



In [20]:
# setup pass index for object index channel
cloth_obj.pass_index = 1

render_cycles = True



if render_cycles:
    # setup output
    setup_cycles(get_cycles_uviz_config())
    setup_color_management_raw()
    # curr_s = time.perf_counter()
    # print("Setup UVIZ: {}".format(curr_s - s))
    # s = curr_s

    # render
    for i in range(num_camera_angles):
        setup_exr_output(uviz_paths[i])
        set_camera_extrinsic(camera_obj, camera_extrinsic_list[i])
        bpy.ops.render.render(animation=False, write_still=True, use_viewport=False)
    # curr_s = time.perf_counter()
    # print("Render UVIZ: {}".format(curr_s - s))
    # s = curr_s

Setting render configuration and printing failed sets.
	denoiser
	sampling_pattern
Fra:0 Mem:152.97M (Peak 152.99M) | Time:00:00.14 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | cloth
Fra:0 Mem:194.60M (Peak 208.27M) | Time:00:00.23 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Initializing
Fra:0 Mem:145.88M (Peak 208.27M) | Time:00:00.23 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Waiting for render to start
Fra:0 Mem:145.88M (Peak 208.27M) | Time:00:00.23 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Loading render kernels (may take a few minutes the first time)
Fra:0 Mem:145.88M (Peak 208.27M) | Time:00:00.27 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Updating Scene
Fra:0 Mem:145.88M (Peak 208.27M) | Time:00:00.27 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Updating Shaders
Fra:0 Mem:145.88M (Peak 208.27M) | Time:00:00.27 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Updating Procedurals
Fra:0 Mem:145.88M (Peak 208.27M) | Time:00:00.27 | Mem:0.00M, Peak:0.00M 

In [21]:
# rgb
# assign materials for rgb
world_material = get_world_material()

setup_hdri_world_material(world_material, hdri_path="../data/hdri/background.exr")

set_material(cloth_obj, cloth_material)

# setup compositor
setup_trivial_compositor()

# setup output
setup_eevee(get_eevee_rgb_config())
setup_color_management_srgb()
# curr_s = time.perf_counter()
# print("Setup RGB: {}".format(curr_s - s))
# s = curr_s

# render
for i in range(num_camera_angles):
    setup_png_output(rgb_paths[i])
    set_camera_extrinsic(camera_obj, camera_extrinsic_list[i])
    bpy.ops.render.render(animation=False, write_still=True, use_viewport=False)
# curr_s = time.perf_counter()
# print("Render RGB: {}".format(curr_s - s))
# s = curr_s

Setting render configuration and printing failed sets.


EGL Error (0x3001): EGL_NOT_INITIALIZED: EGL is not initialized, or could not be initialized, for the specified EGL display connection.
libEGL warning: DRI2: failed to create dri screen

Managed to successfully fallback to surfaceless EGL rendering!

EGL Error (0x3009): EGL_BAD_MATCH: Arguments are inconsistent (for example, a valid context requires buffers not supplied by a valid surface).
EGL Error (0x3001): EGL_NOT_INITIALIZED: EGL is not initialized, or could not be initialized, for the specified EGL display connection.

Managed to successfully fallback to surfaceless EGL rendering!



Fra:0 Mem:186.85M (Peak 186.85M) | Time:00:01.01 | Syncing cloth
Fra:0 Mem:234.82M (Peak 237.11M) | Time:00:01.59 | Syncing main_camera
Fra:0 Mem:234.82M (Peak 237.11M) | Time:00:01.59 | Syncing Empty
Fra:0 Mem:234.82M (Peak 237.11M) | Time:00:01.60 | Rendering 1 / 65 samples
Fra:0 Mem:196.08M (Peak 237.11M) | Time:00:08.36 | Rendering 26 / 64 samples
Fra:0 Mem:196.08M (Peak 237.11M) | Time:00:14.36 | Rendering 51 / 64 samples
Fra:0 Mem:196.08M (Peak 237.11M) | Time:00:17.49 | Rendering 64 / 64 samples
Fra:0 Mem:195.29M (Peak 237.11M) | Time:00:17.73 | Compositing
Fra:0 Mem:195.29M (Peak 237.11M) | Time:00:17.74 | Compositing | Determining resolution
Fra:0 Mem:195.29M (Peak 237.11M) | Time:00:17.74 | Compositing | Initializing execution
Fra:0 Mem:215.36M (Peak 237.11M) | Time:00:17.74 | Compositing | Tile 1-16
Fra:0 Mem:215.36M (Peak 237.11M) | Time:00:17.74 | Compositing | Tile 6-16
Fra:0 Mem:215.36M (Peak 237.11M) | Time:00:17.74 | Compositing | Tile 4-16
Fra:0 Mem:215.36M (Peak 237.

In [22]:
checkpointer.save_checkpoint_if_desired()

Info: Saved "checkpoint_2.blend"
Saved blend file checkpoint to '/home/dcolli23/code/school/rob599_deeprob/projects/final/garmentnets_tracking/simulation/runners/../script_output/sim_pipeline_driver_test/00380_Tshirt_0_rendering_prototyping/checkpoint_2.blend'


2

In [23]:
# pickle non-image data
result_data = {
    'camera': {
        'intrinsic': camera_intrinsic,
        'extrinsic_list': camera_extrinsic_list
    },
    'images': {
        'uviz': [str(pathlib.Path(x).name) for x in uviz_paths],
        'rgb': [str(pathlib.Path(x).name) for x in rgb_paths]
    },
    'meta': {
        'sample_id': sample_id,
        'garment_name': garment_name,
        'gender': gender,
        'fabric': fabric
    }
}
pickle.dump(result_data, open(pickle_path, 'wb'))
curr_s = time.perf_counter()
print("Dump pickle: {}".format(curr_s - s))
s = curr_s

NameError: name 'pathlib' is not defined